<a href="https://colab.research.google.com/github/sgbaird-mwe/mat_discover/blob/main/examples/elmd_densmap_cluster_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
!pip install -U pip
!pip install -UI mat_discover #-UI not necessary when not on Colab

     |████████████████████████████████| 2.1 MB 5.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 59.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.7/543.7 KB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 KB 24.0 MB

In [2]:
"""Load some data, fit Discover(), predict on validation data, and plot."""
# %% imports
from os.path import join
import pandas as pd
from crabnet.data.materials_data import elasticity
from mat_discover.mat_discover_ import Discover
from mat_discover.utils.pareto import pareto_plot

In [3]:
# %% setup
# set dummy to True for a quicker run --> small dataset, MDS instead of UMAP
dummy = False
disc = Discover(dummy_run=dummy, pred_weight=0)
train_df, val_df = disc.data(elasticity, fname="train.csv", dummy=dummy)
train_df # only formula and target columns required

,formula,index,target,count
4002,Li2CuS2,"(11237, 11254)",41.0,2
334,As2O5,"(5126,)",93.0,1
1613,Co2B4Mo,"(8018,)",303.0,1
4329,LiCaF3,"(10205,)",85.0,1
792,Be2CoPt,"(7684,)",207.0,1
...,...,...,...,...
5734,NaPt2,"(2642,)",124.0,1
5191,Mn2GaW,"(7848,)",220.0,1
5390,MnSnPd2,"(6509,)",133.0,1
860,BeNi3,"(11839,)",63.0,1


In [4]:
disc.fit(train_df)

In [5]:
%%time
score = disc.predict(val_df, umap_random_state=42)

val RMSE:  0.0
Fitting mod_petti kernel matrix
Constructing distances
[fit-wasserstein]
Elapsed: 37.73372



/usr/local/lib/python3.7/dist-packages/umap/umap_.py:1780: UserWarning: using precomputed metric; inverse_transform will be unavailable
  warn("using precomputed metric; inverse_transform will be unavailable")


[fit-UMAP]
Elapsed: 115.26902



/usr/local/lib/python3.7/dist-packages/umap/umap_.py:1780: UserWarning: using precomputed metric; inverse_transform will be unavailable
  warn("using precomputed metric; inverse_transform will be unavailable")


[fit-vis-UMAP]
Elapsed: 50.70801

[DensMAP]
Elapsed: 165.97939

[HDBSCAN*]
Elapsed: 0.20407

[pdf-summation]
Elapsed: 15.064

[gridded-pdf-summation]
Elapsed: 15.06753

[train-val-pdf-summation]
Elapsed: 6.64771

[nearest-neighbor-properties]
Elapsed: 3.25226

CPU times: user 3min 36s, sys: 6.82 s, total: 3min 43s
Wall time: 3min 49s


In [6]:
# Interactive scatter plot colored by clusters
x = "DensMAP Dim. 1"
y = "DensMAP Dim. 2"
umap_df = pd.DataFrame(
    {
        x: disc.std_emb[:, 0],
        y: disc.std_emb[:, 1],
        "cluster ID": disc.labels,
        "formula": disc.all_formula,
    }
)
fig = pareto_plot(
    umap_df,
    x=x,
    y=y,
    color="cluster ID",
    fpath=join(disc.figure_dir, "px-umap-cluster-scatter"),
    pareto_front=False,
    parity_type=None,
)